In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/semantic_parsing/tree_slu/main')

In [2]:
%tensorflow_version 1.x
!pip install tensorflow-hub
!pip install texar

     |████████████████████████████████| 266kB 9.1MB/s 
     |████████████████████████████████| 17.4MB 476kB/s 
  Created wheel for texar: filename=texar-0.2.3-cp36-none-any.whl size=395469 sha256=72dcb299e89a8ce749a5a6e0bd527b2739205b8d6ad213b8f1f92aa96ab6fb76
  Stored in directory: /root/.cache/pip/wheels/a9/00/3d/b2c9a2c98e2102bb3fe24ebaa5bd09303fa392aab6f77d78e8
Successfully built texar
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.4
    Uninstalling numpy-1.17.4:
      Successfully uninstalled numpy-1.17.4


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import texar.tf as tx

import numpy as np
import pprint
import logging
import time
import nltk

from texar.tf.modules import TransformerDecoder
from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      #source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (text_tokenized, (target_in, target_out))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ([None], [None]))
  _types = (tf.string, (tf.int32, tf.int32))
  _pads = ('<pad>', (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.repeat(10)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(4*params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def forward(raw_texts, labels, mode):
    vocab = tf.contrib.lookup.index_table_from_file(
        params['vocab_tgt_path'], num_oov_buckets=1)
    words = vocab.lookup(raw_texts)
    words_len = tf.count_nonzero(words, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(words)[0]
    
    
    with tf.variable_scope('Embedding'):
        elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
        e = elmo(inputs={'tokens':raw_texts, 'sequence_len':words_len,}, signature="tokens", as_dict=True)['lstm_outputs1']
        e = tf.layers.dropout(e, params['dropout_rate'], training=is_training)
        e = tf.layers.dense(e, params['rnn_units'], params['activation'])
        
        embedding = tf.Variable(np.load('../vocab/word.npy'),
                                dtype=tf.float32,
                                name='glove')
        x = tf.nn.embedding_lookup(embedding, words)
        x = tf.concat((x, e), -1)
        x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
    
    
    with tf.variable_scope('Encoder'):
        t = tf.transpose(x, perm=[1, 0, 2])  # Need time-major
        lstm_cell_fw = tf.contrib.rnn.LSTMBlockFusedCell(params['rnn_units'])
        lstm_cell_bw = tf.contrib.rnn.LSTMBlockFusedCell(params['rnn_units'])
        lstm_cell_bw = tf.contrib.rnn.TimeReversedFusedRNN(lstm_cell_bw)
        o_fw, s_fw = lstm_cell_fw(t, dtype=tf.float32, sequence_length=words_len)
        o_bw, s_bw = lstm_cell_bw(t, dtype=tf.float32, sequence_length=words_len)
        enc_out = tf.concat([o_fw, o_bw], axis=-1)
        enc_out = tf.transpose(enc_out, perm=[1, 0, 2])
    
    embedding = tf.concat([tf.zeros(shape=[1, 300]), embedding[1:, :]], axis=0)
    pos_embedder = tx.modules.SinusoidsPositionEmbedder(
        position_size = params['max_dec_len'],
        hparams = config_model.position_embedder_hparams)
    
    with tf.variable_scope('Decoder'):
        decoder = TransformerDecoder(vocab_size=len(params['tgt2idx'])+1,
                                     output_layer=tf.transpose(embedding, (1, 0)),
                                     hparams=config_model.decoder)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            dec_inputs, dec_outputs = labels
            dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
            dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
            dec_inputs = (dec_inputs * config_model.hidden_dim ** 0.5) + pos_embedder(sequence_length=dec_seq_len)
            #dec_inputs = tf.layers.dropout(dec_inputs, params['dropout_rate'], training=is_training)

            outputs = decoder(
                memory=enc_out,
                memory_sequence_length=words_len,
                inputs=dec_inputs,
                decoding_strategy='train_greedy',
                mode=tf.estimator.ModeKeys.TRAIN
            )
            
            return outputs.logits
        else:
            start_tokens = tf.fill([batch_sz], 1)

            def _embedding_fn(x, y):
                x_w_embed = tf.nn.embedding_lookup(embedding, x)
                y_p_embed = pos_embedder(y)
                return x_w_embed * config_model.hidden_dim ** 0.5 + y_p_embed

            predictions = decoder(
                memory=enc_out,
                memory_sequence_length=words_len,
                beam_width=params['beam_width'],
                length_penalty=params['length_penalty'],
                start_tokens=start_tokens,
                end_token=2,
                embedding=_embedding_fn,
                max_decoding_length=params['max_dec_len'],
                mode=tf.estimator.ModeKeys.PREDICT)
            
            return predictions['sample_id'][:, :, 0]

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    loss_op = tf.losses.softmax_cross_entropy(onehot_labels = tf.one_hot(dec_outputs, len(params['tgt2idx'])+1),
                                              logits = logits_or_ids,
                                              weights = tf.to_float(tf.sign(dec_outputs)),
                                              label_smoothing = .1)  
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .99)
        """
        lr = (2 * (params['rnn_units'] ** -0.5)) \
            * tf.math.minimum(1.0, tf.to_float(global_step / params['warmup_steps'])) \
            * (1 / tf.math.sqrt(tf.to_float(tf.math.maximum(global_step, params['warmup_steps']))))
        """
        train_op = tf.train.AdamOptimizer(lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': lr}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
params = {
    'model_dir': '../model/lstm_transformer',
    'log_path': '../log/lstm_transformer.txt',
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model/',
    'dropout_rate': 0.2,
    'rnn_units': 300,
    'beam_width': 5,
    'activation': tf.nn.relu,
    'lr': 4e-4,
    'clip_norm': 5.,
    'max_dec_len': 80,
    'length_penalty': .6,
    'buffer_size': 31279,
    'batch_size': 32,
    'num_patience': 5,
}


class config_model:
    hidden_dim = 300

    position_embedder_hparams = {
        'dim': hidden_dim
    }

    decoder = {
        'dim': hidden_dim,
        'num_blocks': 2,
        'multihead_attention': {
            'num_heads': 8,
            'output_dim': hidden_dim,
            'use_bias': True,
        },
        'initializer': {
            'type': 'variance_scaling_initializer',
            'kwargs': {
                'scale': 1.0,
                'mode': 'fan_avg',
                'distribution': 'uniform',
            },
        },
        'poswise_feedforward': tx.modules.default_transformer_poswise_net_hparams(
            output_dim=hidden_dim),
        'output_layer_bias': True,
    }

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
def minimal_test(estimator):
  test_str = [['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']]
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = np.array(test_str), shuffle = False)))
  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str[0]))
  predicted = ' '.join([params['idx2tgt'].get(idx, len(params['idx2tgt'])) for idx in predicted[0]])
  predicted = predicted.replace('<end>', '').replace('<pad>', '').strip()
  print('parsed:', predicted)
  print()
  try:
    nltk.tree.Tree.fromstring(predicted.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [0]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.propagate = False
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
eval_steps = 10 * (params['buffer_size']//params['batch_size'] + 1)
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(save_checkpoints_steps=eval_steps),
  params=params)

best_acc = .0
history_acc = []
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)

  # Evaluation
  labels = [label for _, (_, label) in dataset(is_training=False, params=params)]
  seq_lens = [tf.argmax(tf.cast(tf.equal(l, 2), tf.int32), axis=1) for l in labels]
  labels = [j for i in labels for j in i.numpy()]
  seq_lens = [j for i in seq_lens for j in i.numpy()]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))

  assert len(preds) == len(labels)

  res = [np.all(p[:seq_len+1] == l[:seq_len+1]) for p, l, seq_len in zip(preds, labels, seq_lens)]
  acc = np.asarray(res).mean()
  logger.info("Evaluation: Testing (Exact Match) Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
  logger.info("Best (Exact Match) Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing (Exact Match) Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

INFO:tensorflow:Using config: {'_model_dir': '../model/lstm_transformer', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 9780, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb3bb75b710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/affine_bias:0' shape=(8692,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/beta:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/gamma:0' shape=(300,) dt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/affine_bias:0' shape=(8692,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/beta:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/gamma:0' shape=(300,) dt

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/affine_bias:0' shape=(8692,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/beta:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/gamma:0' shape=(300,) dt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/affine_bias:0' shape=(8692,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/beta:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/gamma:0' shape=(300,) dt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/kernel:0' shape=(900, 1200) dtype=float32_ref>,
 <tf.Variable 'Encoder/lstm_fused_cell_1/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/affine_bias:0' shape=(8692,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/beta:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Decoder/transformer_decoder/layer_0/gamma:0' shape=(300,) dt